# Authenticate

In [ ]:
from google_auth_oauthlib import flow

# The `launch_browser` boolean variable indicates if a local server is used
# as the callback URL in the auth flow. A value of `True` is recommended,
# but a local server does not work if accessing the application remotely,
# such as over SSH or from a remote Jupyter notebook.

appflow = flow.InstalledAppFlow.from_client_secrets_file(
    "credentials.json",
    scopes=["https://www.googleapis.com/auth/bigquery"],
)

launch_browser = True

if launch_browser:
    appflow.run_local_server()
else:
    appflow.run_console()

credentials = appflow.credentials

In [ ]:
from datetime import date

## Client creation

In [ ]:
from google.cloud import bigquery

project = "pypi-analysis-tda2"  # Project ID inserted based on the query results selected to explore
location = "US"  # Location inserted based on the query results selected to explore

client = bigquery.Client(project=project, credentials=credentials, location=location)

# Config

In [ ]:
DAYS = 30

# Most downloaded files

In [ ]:
query_job = client.query(
    f"""
SELECT 
  file.project as project,
  file.version as version,
  DATE(timestamp) as date,
  COUNT(*) as num_downloads
FROM `bigquery-public-data.pypi.file_downloads`
WHERE
  -- avoid bandersnatch and other mirrors
  details.installer.name = 'pip'
  -- use the oldest alive version
  AND details.python LIKE '3.8.%'
  -- use only cpython
  AND details.implementation.name = 'CPython'
  -- Only linux
  AND details.system.name = 'Linux'
  -- Only query the last N days of history
  AND DATE(timestamp)
    BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL {DAYS} DAY)
    AND CURRENT_DATE()
GROUP BY 1, 2, 3
ORDER BY 1 ASC, 2 ASC, 3 ASC
"""
)

results = query_job.to_dataframe(progress_bar_type="tqdm")
results.to_parquet(f"file_downloads__30d__{date.today():%Y%m%d}.parquet")

# Most requested packages

In [ ]:
query_job = client.query(
    f"""
SELECT 
  project,
  DATE(timestamp) as date,
  COUNT(*) as num_requests
FROM `bigquery-public-data.pypi.simple_requests`
WHERE
  -- avoid bandersnatch and the likes
  details.installer.name = 'pip'
  -- use the oldest alive version
  AND details.python LIKE '3.8%'
  -- use only cpython
  AND details.implementation.name = 'CPython'
  -- Only linux
  AND details.system.name = 'Linux'
  -- Only query the last N days of history
  AND DATE(timestamp)
    BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL {DAYS} DAY)
    AND CURRENT_DATE()
GROUP BY 1, 2
ORDER BY 1 ASC, 2 ASC
"""
)

results = query_job.to_dataframe(progress_bar_type="tqdm")
results.to_parquet(f"simple_requests__30d__{date.today():%Y%m%d}.parquet")

# Metadata

In [ ]:
query_job = client.query(
    """
select 
  name,
  version,
  platform,
  requires_python,
  requires,
  provides,
  obsoletes,
  requires_dist,
  provides_dist,
  obsoletes_dist,
  requires_external,
  upload_time,
  filename,
  python_version
from `bigquery-public-data.pypi.distribution_metadata`
;
"""
)

In [ ]:
results = query_job.to_dataframe(progress_bar_type="tqdm")

In [ ]:
results.to_parquet(f"metadata__{date.today():%Y%m%d}.parquet")